In [11]:
import cv2
import numpy as np
import scipy.ndimage.filters as filters
from scipy import signal
import scipy.ndimage as ndimage
import sys

In [10]:
def hough(path, my_theta, threshold):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape
    filter_size = 10 
    dedc_angle = 90
    pixel_max = 255

    ed_img = cv2.Canny(img, 100, 200)
#     which shows the edges of the img
    cv2.imshow('edge', ed_img)
    cv2.imwrite('output/edge.png', ed_img)
    cv2.waitKey()
    
    _m = np.linspace(-dedc_angle, dedc_angle, my_theta)
    thetas = np.radians(_m)
    _t = int(np.sqrt(h**2 + w**2))
    rhos = np.linspace(_t, -_t, _t * 5)

    cos = np.cos(thetas)
    sin = np.sin(thetas)
    num_thetas = len(thetas)

    cur_kernel = np.zeros((num_thetas, 5 * _t))

    for i in range(h):
        for j in range(w):
            if ed_img[i][j] > 0:
                for theta in range(num_thetas):
                    rho = int(round(i * cos[theta] + j * sin[theta]) + _t)
                    cur_kernel[theta][rho] += 1

    cur_kernel_img = cur_kernel / np.max(cur_kernel) * pixel_max
    
    cv2.imshow('cur_kernel',  cur_kernel_img.astype(np.uint8))
    cv2.imwrite('output/my_theta.png',  cur_kernel_img.astype(np.uint8))
    cv2.waitKey()

    h_, w_ = cur_kernel.shape
    max_num = np.max(cur_kernel)
    thresh_img = np.zeros(cur_kernel.shape)

    for i in range(h_):
        for j in range(w_):
            if int(cur_kernel[i][j]) >= threshold * max_num:
                thresh_img[i][j] = pixel_max

    cv2.imshow('threshold', thresh_img)
    cv2.imwrite('output/threshold_my_theta.png', thresh_img)
    cv2.waitKey()

    max_ = filters.maximum_filter(cur_kernel, filter_size)
    min_ = filters.minimum_filter(cur_kernel, filter_size)
    max_ker = (cur_kernel == max_)
    max_ker[((max_ - min_) > threshold) == 0] = 0

    labeled, num_objects = ndimage.label(max_ker)
    slices = ndimage.find_objects(labeled)

    x, y = [], []
    for dy,dx in slices:
        x_center = (dx.start + dx.stop - 1) / 2
        x.append(x_center)
        y_center = (dy.start + dy.stop - 1) / 2
        y.append(y_center)

    max_ker_col = np.zeros(cur_kernel.shape)
    for i in range(len(x)):
        max_ker_col[int(y[i]),int(x[i])] = pixel_max

#     cv2.imshow('max_nd', max_ker_col)
#     cv2.imwrite('output/nd for.png', max_ker_col)
#     cv2.waitKey()
    
    cut_img = ed_img
    h__, w__ = img.shape
    for i, j in zip(y__, x__):
        theta = thetas[int(i)]
        rho = rhos[int(j)]

        if (np.degrees(theta)) > -45 and (np.degrees(theta) < 45):
            for y in range(h__):
                x = int((-y * np.sin(theta) + rho) / np.cos(theta))
                if x in range(w__):
                    cut_img[y][x] = pixel_max
        else:
            for x in range(w__):
                y = int((-x * np.cos(theta) + rho) / np.sin(theta))
                if y in range(h__):
                    cut_img[y][x] = pixel_max
    cv2.line(cut_img, (0, 0), (1, 1), (pixel_max,0,0), thickness = 10)
    cv2.imshow('cut_img', cut_img)
    cv2.imwrite('output/cut_img.png', cut_img)
    cv2.waitKey()
    
#   ==============================================================================
    cv2.destroyAllWindows()
    return 0

In [ ]:
# path = 'input/test.bmp'
# my_img_1 = hough(path, 355, threshold = 0.9)

# path = 'input/test2.bmp'
# my_img_2 = hough(path, 355, threshold = 0.9)

path = 'input/input.bmp'
my_img_3 = hough(path, 355, threshold = 0.9)
